In [ ]:
# 導入函式庫以及讀取資料
import pandas as pd


df = pd.read_csv("water_potability.csv")

In [ ]:
# 特徵分布分析
from matplotlib import pyplot as plt


df.hist(figsize=(15, 15))
plt.show()

In [ ]:
# 分析特徵相依性
# 正數為有正關聯，負數則為負關聯
import numpy as np
import seaborn as sns

plt.figure(figsize=(8, 8))
corrMask = np.triu(df.corr())
sns.heatmap(
    df.corr(),
    linewidths=1,
    annot=True,
    square=True,
    mask=corrMask,
    cmap="Blues",
)
plt.show()

In [ ]:
# 填補空缺值
empty = df.columns[df.isna().any()].tolist()
print("有空缺的資料：", empty)
print("填補空缺值...")
for i in empty:
    df[i].fillna(value=df[i].median(), inplace=True)
print("確認填補結果：")
df.isnull().sum()

In [ ]:
# 切分特徵和目標變量
from sklearn.model_selection import train_test_split


X = df.drop("Potability", axis=1)
y = df["Potability"]
# 切分訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75)

In [ ]:
# 決策樹
# 對水質數據進行分類
"""
在機器學習中，超參數是在模型擬合之前需要手動設定的參數，而不是由模型自動學習的參數。
決策樹有一些超參數，例如樹的深度 (max_depth) 和每個節點最大特徵數 (max_features) 等，這些超參數的設定會影響模型的性能。
通常情況下，我們不知道哪組超參數值是最佳的，因此我們使用網格搜索來通過在指定的超參數範圍內嘗試不同的組合，從而找到最佳的超參數組合。
這樣做的好處是，可以自動化地找到最佳超參數，而不需要手動進行不斷的調整。這有助於提高模型的性能，使其更能適應數據並提高預測能力。
"""
"""
這裡使用網格搜索 (GridSearchCV) 來尋找最佳的 K 值。
定義了一個 K 值的範圍（1 到 101），然後使用網格搜索在這個範圍內尋找最佳的 K 值。
GridSearchCV 通過交叉驗證（cv=5）來評估每個 K 值的性能。
最後，印出最佳的 K 值和對應的交叉驗證分數。
"""

# 定義決策樹模型的超參數範圍
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree


gridSearchParam = {
    "max_depth": [3],
    "max_features": [i for i in range(1, len(X.columns) + 1)],
}

# 使用網格搜索(GridSearchCV)尋找最佳超參數
gridSearch = GridSearchCV(DecisionTreeClassifier(), gridSearchParam, cv=5)
gridSearch.fit(X_train, y_train)
print(gridSearch.best_params_)  # 輸出最佳超參數
print(gridSearch.best_score_)  # 輸出最佳模型的交叉驗證分數
best_param = gridSearch.best_params_

# 建立決策樹模型
dt = DecisionTreeClassifier(max_depth=3, max_features=best_param["max_features"])
dt.fit(X_train, y_train)

# 進行預測
dtPredict = dt.predict(X_test)

# 評估模型性能

dtAccuracy = accuracy_score(y_test, dtPredict)
print(f"模型準確率：{dtAccuracy:.2f}")

# 分類報告
print("DecisionTree Classification Report: ")
print(classification_report(y_test, dtPredict))

# 混淆矩陣
# 使用Seaborn來繪製熱力圖
plt.figure(figsize=(5, 3))
sns.heatmap(
    confusion_matrix(y_test, dtPredict),
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Non-Potable", "Potable"],
    yticklabels=["Non-Potable", "Potable"],
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("DecisionTree Confusion Matrix")
plt.show()

# 可視化決策樹
plt.figure(figsize=(15, 10))
plot_tree(
    dt,
    feature_names=X.columns.tolist(),
    class_names=["Non-Potable", "Potable"],
    filled=True,
    rounded=True,
    fontsize=10,
)
plt.show()

# 交叉驗證，評估模型的穩健性
crossVal = cross_val_score(dt, X, y, cv=5)
print(f"Cross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

plt.bar(X.columns, dt.feature_importances_)
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.xticks(rotation=45, ha="right")
plt.title("Random Forest Feature Importance")
plt.show()

In [ ]:
# RandomForest
# 創建一個隨機森林分類器
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練模型
rf.fit(X_train, y_train)

# 使用測試集進行預測
rfPredict = rf.predict(X_test)

# 評估模型準確率
rfAccuracy = accuracy_score(y_test, rfPredict)
print(f"模型準確率：{rfAccuracy:.2f}")

# 列印分類報告
print("分類報告:")
print(classification_report(y_test, rfPredict))

# 混淆矩陣
# 顯示混淆矩陣
plt.figure(figsize=(8, 6))
sns.heatmap(
    confusion_matrix(y_test, rfPredict),
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Not Potable", "Potable"],
    yticklabels=["Not Potable", "Potable"],
)
plt.title("Random Forest Confusion Matrix")
plt.xlabel("Prediction")
plt.ylabel("Test Data")
plt.show()


plt.bar(X.columns, rf.feature_importances_)
plt.xticks(rotation=45, ha="right")
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.title("SVM Coefficient")
plt.show()


# 交叉驗證，評估模型的穩健性
crossVal = cross_val_score(rf, X, y, cv=5)
print(f"Cross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

In [ ]:
# 邏輯回歸
from sklearn.linear_model import LogisticRegression


lr = LogisticRegression()
lr.fit(X_train, y_train)
lrPredict = lr.predict(X_test)

# 評估模型準確率
lrAccuracy = accuracy_score(y_test, lrPredict)
print(f"模型準確率：{lrAccuracy:.2f}")

print("分類報告:")
print(classification_report(y_test, lrPredict))

plt.figure(figsize=(8, 6))
sns.heatmap(
    confusion_matrix(y_test, lrPredict),
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Not Potable", "Potable"],
    yticklabels=["Not Potable", "Potable"],
)
plt.title("Logistic Return Confusion Matrix")
plt.xlabel("Prediction")
plt.ylabel("Test Data")
plt.show()

plt.bar(X.columns, lr.coef_[0])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Feature")
plt.ylabel("Coefficient")
plt.title("Logistic Regression Coefficient")
plt.show()


# 交叉驗證，評估模型的穩健性
crossVal = cross_val_score(lr, X, y, cv=5)
print(f"Cross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

In [ ]:
# BaggingClassifier
from sklearn.ensemble import BaggingClassifier


bagging = BaggingClassifier()
bagging.fit(X_train, y_train)
baggingPredict = bagging.predict(X_test)


baggingAccuracy = accuracy_score(y_test, baggingPredict)
print(f"模型準確率：{baggingAccuracy:.2f}")
print("KNN Classification Report:")
print(classification_report(y_test, baggingPredict))

# 使用Seaborn來繪製混淆矩陣
plt.figure(figsize=(5, 3))
sns.heatmap(
    confusion_matrix(y_test, baggingPredict),
    annot=True,
    fmt="d",
    cmap="YlOrBr",
    cbar=False,
    xticklabels=["Non-Potable", "Potable"],
    yticklabels=["Non-Potable", "Potable"],
)
plt.xlabel("Prediction")
plt.ylabel("Test Data")
plt.title("Bagging Classifier Confusion Matrix")
plt.show()

# 交叉驗證
crossVal = cross_val_score(bagging, X, y, cv=5)
print(f"\nCross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

In [ ]:
# KNN

# 特徵標準化
from sklearn.discriminant_analysis import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

"""
這裡使用網格搜索 (GridSearchCV) 來尋找最佳的 K 值。
定義了一個 K 值的範圍（1 到 21），然後使用網格搜索在這個範圍內尋找最佳的 K 值。
GridSearchCV 通過交叉驗證（cv=5）來評估每個 K 值的性能。
最後，印出最佳的 K 值和對應的交叉驗證分數。
"""

# 定義決策樹模型的超參數範圍
gridSearchParam = {"n_neighbors": range(1, 21)}

# 使用網格搜索尋找最佳K值
gridSearch = GridSearchCV(KNeighborsClassifier(), gridSearchParam, cv=5)
gridSearch.fit(X_train_scaled, y_train)
print(gridSearch.best_params_)
print(gridSearch.best_score_)
best_param = gridSearch.best_params_["n_neighbors"]

# 使用最佳K值擬合新的KNN模型
knn = KNeighborsClassifier(n_neighbors=best_param)
knn.fit(X_train_scaled, y_train)

# 預測
knnPredict = knn.predict(X_test_scaled)

# 評估模型性能 分類報告

knnAccuracy = accuracy_score(y_test, knnPredict)
print(f"模型準確率：{knnAccuracy:.2f}")
print("KNN Classification Report:")
print(classification_report(y_test, knnPredict))

# 使用Seaborn來繪製混淆矩陣
plt.figure(figsize=(5, 3))
sns.heatmap(
    confusion_matrix(y_test, knnPredict),
    annot=True,
    fmt="d",
    cmap="YlOrBr",
    cbar=False,
    xticklabels=["Non-Potable", "Potable"],
    yticklabels=["Non-Potable", "Potable"],
)
plt.xlabel("Prediction")
plt.ylabel("Test Data")
plt.title("KNN Confusion Matrix")
plt.show()

# 交叉驗證
crossVal = cross_val_score(knn, X, y, cv=5)
print(f"\nCross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

In [ ]:
# SVM
# 初始化 SVM 分類器
from sklearn.svm import SVC


svm = SVC()

# 訓練 SVM 模型
svm.fit(X_train_scaled, y_train)

# 使用測試集進行預測
svmPredict = svm.predict(X_test_scaled)

# 評估模型準確率
svmAccuracy = accuracy_score(y_test, svmPredict)
print(f"模型準確率：{svmAccuracy:.2f}")

# 顯示分類報告
print("分類報告：")
print(classification_report(y_test, svmPredict))

# 混淆矩陣

# 顯示混淆矩陣
plt.figure(figsize=(8, 6))
sns.heatmap(
    confusion_matrix(y_test, svmPredict),
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Not Potable", "Potable"],
    yticklabels=["Not Potable", "Potable"],
)
plt.title("SVM Confusion Matrix")
plt.xlabel("Prediction")
plt.ylabel("Test Data")
plt.show()
"""
# 顯示特徵重要性
plt.bar(X.columns, svm.coef_[0])
plt.xticks(rotation=45, ha="right")
plt.xlabel("Features")
plt.ylabel("Coefficient")
plt.title("SVM Coefficient")
plt.show()
"""

# 交叉驗證，評估模型的穩健性
crossVal = cross_val_score(svm, X, y, cv=5)
print(f"Cross-Validation Scores: {crossVal}")
print(f"Mean Accuracy: {crossVal.mean():.2f}")

In [ ]:
scores = pd.DataFrame(
    {
        "models": [
            "Logistic Regression",
            "Decision Tree",
            "Random Forest",
            "KNN",
            "SVM",
            "Bagging Classifier",
        ],
        "accuracy": [
            lrAccuracy,
            dtAccuracy,
            rfAccuracy,
            knnAccuracy,
            svmAccuracy,
            baggingAccuracy,
        ],
    }
)
scores.sort_values(by=["accuracy"]).style.background_gradient(subset=["accuracy"])